In [1]:
!pip install h5py

In [1]:
import h5py
h5f = h5py.File('./SVHN_single_grey1.h5', 'r')

# Load the training and the test set
X_train = h5f['X_train'][:]
y_train = h5f['y_train'][:]
X_test = h5f['X_test'][:]
y_test = h5f['y_test'][:]


# Close this file
h5f.close()

In [2]:
len(X_train), len(X_test),X_train.shape, X_test.shape

(42000, 18000, (42000, 32, 32), (18000, 32, 32))

In [3]:
X_train = X_train.reshape(X_train.shape[0], 1024)
X_test = X_test.reshape(X_test.shape[0], 1024)
# Normalize inputs from 0-255 to 0-1

X_train = X_train/255
X_test = X_test/255

In [4]:
svh_x_train, svh_x_test, svh_y_train, svh_y_test = X_train, X_test, y_train, y_test

In [5]:
from sklearn import model_selection

svh_X_train, svh_x_val, svh_Y_train, svh_y_val = model_selection.train_test_split(svh_x_train, svh_y_train, test_size=1/3, random_state=42)

In [6]:
from util import Utils


Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [14]:
best_svh_model = Utils.find_best_rf_parameters(svh_X_train, svh_Y_train, svh_x_val, svh_y_val)

|   n_estimators | criterion   |   max_depth | max_features   |   score in validation set |
|---------------:|:------------|------------:|:---------------|--------------------------:|
|            100 | gini        |         100 | log2           |                  0.679714 |
|             50 | gini        |         100 | log2           |                  0.639286 |
|             10 | gini        |         100 | log2           |                  0.478143 |
|            100 | gini        |         100 | sqrt           |                  0.706429 |
|             50 | gini        |         100 | sqrt           |                  0.678571 |
|             10 | gini        |         100 | sqrt           |                  0.539857 |
|            100 | gini        |          50 | log2           |                  0.685214 |
|             50 | gini        |          50 | log2           |                  0.640571 |
|             10 | gini        |          50 | log2           |                 

In [15]:
Utils.accuracy_measure_rf(svh_x_train, svh_y_train, 
                          svh_x_test, svh_y_test, 
                          n_estimators = 100, criterion = 'entropy', max_depth = 50, max_features = 'sqrt')

--- 439.25284361839294 seconds ---


(0.7373333333333333,
 (array([0.71400297, 0.69835294, 0.79467456, 0.6676009 , 0.75741801,
         0.73447016, 0.73675214, 0.80928412, 0.73873874, 0.73372093]),
  array([0.7954796 , 0.81181619, 0.74486966, 0.69284468, 0.80298013,
         0.6821267 , 0.70578603, 0.80033186, 0.63355408, 0.69955654]),
  array([0.75254237, 0.75082216, 0.7689665 , 0.67998858, 0.77953389,
         0.70733138, 0.72093672, 0.80478309, 0.68211527, 0.71623156]),
  array([1814, 1828, 1803, 1719, 1812, 1768, 1832, 1808, 1812, 1804])))

In [ ]:
best_xgb_model = Utils.find_best_xgb_parameters(svh_X_train, svh_Y_train, svh_x_val, svh_y_val)

Finished {'learning_rate': 0.01, 'max_depth': 10, 'n_estimators': 10}
Finished {'learning_rate': 0.01, 'max_depth': 10, 'n_estimators': 100}
